# Setup and run Bayesian inference on a batch of GW events using `pycbc_inference`

## Setup

In [13]:
import os

run_dir = '/home/prayush/research/test_pycbc_gw150914'
try: os.makedirs(run_dir)
except: pass
os.chdir(run_dir)

In [7]:
!pwd

/home/prayush/research/test_pycbc_gw150914


In [8]:
!rm -rf *

In [9]:
!ls

## Generate sampler / inference config files

In [16]:
!gwnr_write_pycbc_inference_configs -h

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_write_pycbc_inference_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a set of signals with Pycbc inference

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-data-config WRITE_DATA_CONFIG
                        Write data config files an

In [17]:
!gwnr_write_pycbc_inference_configs --show-available-configs

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
data: ['gw150914-like-gaussian', 'gw150914-like-zeronoise', 'GW150914-v3', 'GW151012-v3', 'GW151226-v2', 'GW170104-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v3', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1']
sampler: ['emcee', 'emcee_pt', 'epsie', 'dynesty', 'ultranest', 'multinest', 'cpnest']
inference: ['bbh_precessing', 'bbh_alignedspin']


In [18]:
!gwnr_write_pycbc_inference_configs --verbose\
  --write-sampler-config emcee_pt --write-inference-config bbh_precessing

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
2021-09-20 19:37:49,110 Writing config file for sampler settings..
2021-09-20 19:37:49,111 Writing config file for inference settings..
2021-09-20 19:37:49,111 Done


In [11]:
!ls

bbh_precessing.ini  emcee_pt.ini


In [12]:
!cat emcee_pt.ini bbh_precessing.ini

[sampler]
name = emcee_pt
nprocesses = 10
nwalkers = 1000
ntemps = 20
effective-nsamples = 4000
max-samples-per-chain = 1000
checkpoint-interval = 2000

[sampler-burn_in]
burn-in-test = nacl & max_posterior

;
;   Sampling transforms
;
[sampling_params]
; parameters on the left will be sampled in
; parametes on the right
mass1, mass2 : mchirp, q

[sampling_transforms-mchirp+q]
; inputs mass1, mass2
; outputs mchirp, q
name = mass1_mass2_to_mchirp_q
[model]
name = gaussian_noise
low-frequency-cutoff = 20.0

[variable_params]
; waveform parameters that will vary in MCMC
delta_tc =
mass1 =
mass2 =
spin1_a =
spin1_azimuthal =
spin1_polar =
spin2_a =
spin2_azimuthal =
spin2_polar =
distance =
coa_phase =
inclination =
polarization =
ra =
dec =

[static_params]
; waveform parameters that will not change in MCMC
approximant = IMRPhenomPv2
f_lower = 20
f_ref = 20
; we'll set the tc by using the trigger time in the data
; section of the config file + delta_tc
trigger_time = ${data|trigger-time}

## Write workflow config

In [7]:
with open('config.ini', 'w') as fout:
    fout.write('''\
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
sampler = emcee_pt.ini
inference = bbh_precessing.ini
events = GW150914 GW170104
sample-rate = 2048
data-sample-rate = 4096
data-duration = 4096
psd-estimation = download ; or data-standard

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
output-file = inference.hdf
nprocesses = 10
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Visualize
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[plot]
input-file = inference.hdf
plot-scatter =
plot-marginal =
plot-prior = inference.ini data.ini

[plot-mass1mass2]
output-file = plots/posteriors.png
parameters = 'mass1 mass2'
''')

In [8]:
!cat config.ini

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
sampler = emcee_pt.ini
inference = bbh_precessing.ini
events = GW150914 GW170104
sample-rate = 2048
data-sample-rate = 4096
data-duration = 4096
psd-estimation = download ; or data-standard

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
output-file = inference.hdf
nprocesses = 10
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Visualize
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[plot]
input-file = inference.hdf
plot-scatter =
plot-marginal =
plot-prior = inference.ini data.ini

[plot-mass1mass2]
output-file = plots/posterio

## Generate workflow

In [23]:
!gwnr_create_public_events_pycbc_inference_workflow -h

usage: /home/prayush/src/GWNRTools/bin//gwnrtools_create_public_events_inference_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of public gravitational-wave events using open data

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --output-dir OUTPUT_DIR
                        Output directory path.
  --force               If the output-dir already exists, overwrite it.
                        Otherwise, an OSError is raised.
  --do-not-fetch-data   Don't fetch GWOSC data.
  --nprocesses NPROCESSES
                        Number of processes to use. If not given then only a
                        single core will be used.
  --use-mpi             Use MPI to parallelize the sampler
  --seed SEED           Seed to use for the random number generator that
                        initially distributes the walker

In [3]:
!gwnr_create_public_events_pycbc_inference_workflow --config-files config.ini --output-dir . --force --verbose

2020-03-17 17:49:18,790 Reading configuration file
2020-03-17 17:49:18,792 Using seed 0
2020-03-17 17:49:18,794 Running with CPU support: 1 threads
2020-03-17 17:49:18,904 Will setup analyses in .
2020-03-17 17:49:18,906 Making workspace directories
2020-03-17 17:49:18,907 Creating DAG
2020-03-17 17:49:18,914 Making eventGW170104/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_gw150914
2020-03-17 17:49:18,984 Copying config files to eventGW170104/emcee_pt/bbh_precessing
2020-03-17 17:49:18,986 Copying executables to eventGW170104/emcee_pt/bbh_precessing/scripts/
2020-03-17 17:49:18,988 Fetching GWOSC frame data


2020-03-17 17:50:43,606 Fetching PSD files


2020-03-17 17:51:37,611 Making eventGW150914/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_gw150914
2020-03-17 17:51:37,701 Copying config files to eventGW150914/emcee_pt/bbh_precessing
2020-03-17 17:51:37,704 Copying executables to eventGW150914/emcee_pt/bbh_precessing/scripts/
2020-03-17 17:51:37,706 Fetching GWOSC frame data


2020-03-17 17:52:54,887 Fetching PSD files


2020-03-17 17:53:50,550 Done


In [5]:
!tree

.
├── bbh_precessing.ini
├── config.ini
├── emcee_pt.ini
├── eventGW150914
│   ├── data
│   │   ├── H-H1_GWOSC_4KHZ_R1-1126257415-4096.gwf
│   │   ├── L-L1_GWOSC_4KHZ_R1-1126257415-4096.gwf
│   │   ├── psd_H1.dat
│   │   └── psd_L1.dat
│   └── emcee_pt
│       └── bbh_precessing
│           ├── data.ini
│           ├── inference.hdf.bkup
│           ├── inference.hdf.checkpoint
│           ├── inference.ini
│           ├── log
│           │   ├── run_inference-166-0.err
│           │   ├── run_inference-166-0.out
│           │   ├── run_inference-169-0.err
│           │   ├── run_inference-169-0.out
│           │   ├── run_inference-173-0.err
│           │   ├── run_inference-173-0.out
│           │   └── tmpFgLflx
│           ├── make_plot_mass1mass2
│           ├── make_plot_mass1mass2.sub
│           ├── plots
│           ├── run_inference
│           ├── run_inference.sub
│           ├── sampler.ini
│           └── scripts
│               ├── pycbc_inference
│               └── pyc

## Submit & monitor workflow

In [16]:
!condor_submit_dag pycbc_inference_events.dag >> dag.out

In [17]:
!tail *dagman.out

03/04/20 14:44:41 Workflow accounting_group_user: <>
03/04/20 14:44:41 Warning: failed to get attribute DAGNodeName
03/04/20 14:44:41 DAGMAN_LOG_ON_NFS_IS_ERROR setting: False
03/04/20 14:44:41 Default node log file is: </home/prayush/research/test_pycbc_gw150914/./pycbc_inference_events.dag.nodes.log>
03/04/20 14:44:41 DAG Lockfile will be written to pycbc_inference_events.dag.lock
03/04/20 14:44:41 DAG Input file is pycbc_inference_events.dag
03/04/20 14:44:41 Parsing 1 dagfiles
03/04/20 14:44:41 Parsing pycbc_inference_events.dag ...
03/04/20 14:44:41 Dag contains 4 total jobs
03/04/20 14:44:41 Sleeping for 3 seconds to ensure ProcessId uniqueness


# Setup and run Bayesian inference on a batch of GW events using `bilby`

## Setup

In [12]:
import os

run_dir = '/home/prayush/research/test_bilby_events'
try: os.makedirs(run_dir)
except: pass
os.chdir(run_dir)

In [3]:
!pwd

/home/prayush/research/test_bilby_events


In [4]:
!rm -rf *

In [5]:
!ls

## Generate workflow config files

In [10]:
!gwnr_write_bilby_configs --help

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_write_bilby_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a custom set of signals with Bilby

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-config WRITE_CONFIG
                        Comma-separated name of config-type,config-subtyp

In [11]:
!gwnr_write_bilby_configs --show-available-configs

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
prior: ['default', 'eccentric-nonspin', 'precessing_spins_bns', 'GW170817', 'GW150914', 'precessing_spins_bbh_tides_on', 'aligned_spins_bbh_tides_on', 'precessing_spins_bns_tides_on', 'aligned_spins_bbh', 'precessing_spins_bbh', 'aligned_spins_bns', 'aligned_spins_bns_tides_on']
injection: ['bbh-prior-default', 'bbh-alignedspin-prior-default', 'bbh-eccentric-nonspin-prior-default', 'bbh-prior-eccentric-nonspin', 'bbh-alignedspin-prior-eccentric-nonspin', 'bbh-eccentric-nonspin-prior-eccentric-nonspin', 'bbh-prior-precessing_spins_bns', 'bbh-alignedspin-prior-precessing_spins_bns', 'bbh-

In [14]:
!gwnr_write_bilby_configs --write-config event,bbh-event-prior-GW150914 --verbose

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
2021-09-20 19:37:00,651 Done


In [15]:
!ls

bbh-event-prior-GW150914.ini


In [25]:
!cat bbh-event-prior-GW150914.ini


[workflow]
tag = test
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
request-memory = 2G
request-cpus = 1

[inference]
duration = 4
sample_rate = 2048
lower_frequency_cutoff = 30
upper_frequency_cutoff = 1024
reference_frequency = 30
phase_marginalization =
time_marginalization =
;distance_marginalization =

[sampler]
name = dynesty
npoints = 2000
maxmcmc = 2000
n_check_point = 1000

[data]
analysis_type = event
source_type = bbh
event_names = GW150914,GW170104
interferometers = H1,L1


[template]
source_model = bilby.gw.source.lal_binary_black_hole
approximant = IMRPhenomPv2
sample_rate = 2048
lower_frequency_cutoff = 30
upper_frequency_cutoff = 1024
reference_frequency = 30


[prior]
mass_ratio = Uniform(name='mass_ratio', minimum=0.125, maximum=1)
chirp_mass = Uniform(name='chirp_mass', minimum=25, maximum=31)
mass_1 = Constraint(name='mass_1', minimum=10, maximum=80)
mass_2 = Constraint(name='mass_2', minimum=10, maximum=80)
a_1 = Uniform(name='a_1', minimum=0, max

## Generate workflow

In [19]:
!gwnr_create_public_events_bilby_workflow --help

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_create_public_events_bilby_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of simulated signals

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --config-file CONFIG_FILE
                        Configuration file with details of analyses. See
                        `gwnrtoo

In [22]:
!gwnr_create_public_events_bilby_workflow --config-file bbh-event-prior-GW150914.ini --output-dir bbh-events --verbose

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
2021-09-20 19:42:43,113 Using seed 0
2021-09-20 19:42:43,130 Will setup analyses in bbh-events
2021-09-20 19:42:43,130 Creating DAG
2021-09-20 19:42:43,130 --- creating script writer objects for events
2021-09-20 19:42:43,131 --- script writer object created for event GW150914
2021-09-20 19:42:43,131 --- analysis objects created for event 0
2021-09-20 19:42:43,131 --- script writer object created for event GW170104
2021-09-20 19:42:43,131 --- analysis objects created for event 1
2021-09-20 19:42:43,131 Making ./GW150914 in /home/prayush/research/test_pycbc_gw150914/bbh-events
Writing sc

In [24]:
!tree

.
├── bbh-event-prior-GW150914.ini
├── bbh-events
│   ├── bbh-event-prior-GW150914.ini
│   ├── bilby_events.dag
│   ├── bilby_events.sh
│   ├── GW150914
│   │   ├── log
│   │   ├── priors.prior
│   │   ├── run_inference
│   │   └── run_inference.sub
│   └── GW170104
│       ├── log
│       ├── priors.prior
│       ├── run_inference
│       └── run_inference.sub
├── bbh_precessing.ini
└── emcee_pt.ini

5 directories, 12 files


## Submit & monitor workflow

In [28]:
os.chdir('bbh-events')
!ls

bbh-event-prior-GW150914.ini  bilby_events.sh  GW170104
bilby_events.dag	      GW150914


In [29]:
!condor_submit_dag bilby_events.dag >> dag.out

In [30]:
!tail *dagman.out

09/20/21 19:46:08 Number of idle job procs: 1
09/20/21 19:46:08 Reassigning the id of job 23e9b16444d8cf36712e2be1b5c18c7d from (112.0.0) to (112.0.0)
09/20/21 19:46:08 Event: ULOG_SUBMIT for HTCondor Node 23e9b16444d8cf36712e2be1b5c18c7d (112.0.0) {09/20/21 19:46:08}
09/20/21 19:46:08 Number of idle job procs: 2
09/20/21 19:46:08 DAG status: 0 (DAG_STATUS_OK)
09/20/21 19:46:08 Of 2 nodes total:
09/20/21 19:46:08  Done     Pre   Queued    Post   Ready   Un-Ready   Failed
09/20/21 19:46:08   ===     ===      ===     ===     ===        ===      ===
09/20/21 19:46:08     0       0        2       0       0          0        0
09/20/21 19:46:08 0 job proc(s) currently held
